In [7]:
import pygame
import random

# Инициализация Pygame
pygame.init()

# Размеры окна
WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Road Runner Game")

# Изображения
image_background = pygame.image.load('resources/AnimatedStreet.png')
image_player = pygame.image.load('resources/Player.png')
image_enemy = pygame.image.load('resources/Enemy.png')
image_coin = pygame.image.load('resources/Coin.png')
image_coin = pygame.transform.scale(image_coin, (50, 50))

# Звуки
pygame.mixer.music.load('resources/background.wav')
pygame.mixer.music.play(-1)
sound_crash = pygame.mixer.Sound('resources/crash.wav')
sound_coin = pygame.mixer.Sound('resources/coin.mp3')

# Шрифты
font_large = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
image_game_over = font_large.render("Game Over", True, "black")

# Цвета и настройки
WHITE = (255, 255, 255)
FPS = 60
clock = pygame.time.Clock()

# Классы
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = image_player
        self.rect = self.image.get_rect(center=(WIDTH // 2, HEIGHT - 50))
        self.speed = 5

    def move(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            self.rect.move_ip(-self.speed, 0)
        if keys[pygame.K_RIGHT]:
            self.rect.move_ip(self.speed, 0)
        self.rect.clamp_ip(screen.get_rect())

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = image_enemy
        self.rect = self.image.get_rect()
        self.speed = random.randint(5, 10)
        self.reset_position()

    def reset_position(self):
        self.rect.x = random.randint(0, WIDTH - self.rect.width)
        self.rect.y = -self.rect.height

    def move(self):
        self.rect.move_ip(0, self.speed)
        if self.rect.top > HEIGHT:
            self.reset_position()

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = image_coin
        self.rect = self.image.get_rect()
        self.weight = random.choice([1, 2, 5])
        self.reset_position()

    def reset_position(self):
        self.rect.x = random.randint(0, WIDTH - self.rect.width)
        self.rect.y = random.randint(-200, -50)

    def move(self):
        self.rect.move_ip(0, 5)
        if self.rect.top > HEIGHT:
            self.reset_position()

# Game Over
def show_game_over():
    screen.fill("red")
    screen.blit(image_game_over, (WIDTH // 2 - image_game_over.get_width() // 2, HEIGHT // 2))
    pygame.display.flip()
    pygame.mixer.music.stop()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                raise SystemExit()
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                waiting = False

    pygame.mixer.music.play(-1)

# Текст
def draw_text(text, font, color, x, y):
    img = font.render(text, True, color)
    screen.blit(img, (x, y))

# Основная игра
def main():
    player = Player()
    enemies = pygame.sprite.Group(Enemy())
    coins = pygame.sprite.Group(Coin(), Coin())
    all_sprites = pygame.sprite.Group(player, *enemies, *coins)

    coin_count = 0
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                return

        player.move()
        for entity in all_sprites:
            entity.move()

        if pygame.sprite.spritecollideany(player, enemies):
            sound_crash.play()
            show_game_over()
            return  # выход из main

        coins_collected = pygame.sprite.spritecollide(player, coins, True)
        for coin in coins_collected:
            sound_coin.play()
            coin_count += coin.weight
            new_coin = Coin()
            coins.add(new_coin)
            all_sprites.add(new_coin)

        if coin_count % 10 == 0 and coin_count > 0:
            for enemy in enemies:
                enemy.speed += 0.3

        screen.blit(image_background, (0, 0))
        for entity in all_sprites:
            screen.blit(entity.image, entity.rect)

        draw_text(f"Score: {coin_count * 10}", font_small, WHITE, 10, 10)
        draw_text(f"Coins: {coin_count}", font_small, WHITE, WIDTH - 100, 10)

        pygame.display.flip()
        clock.tick(FPS)

# Запуск
main()


SystemExit: 